In [1]:
import json
import re
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.corpus import stopwords

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Removing Stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#Visualization
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


/home/ubuntu/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Read json files and convert it to csv file

In [4]:
csv_file = open('review_extracted.csv', 'w')
csv_file.write('review_id,user_id,business_id,stars,text,date\n')
with open('yelp_dataset/review.json', 'r') as review_json:
    line = review_json.readline()
    while line:
        json_object = json.loads(line)
        one_entry = str()
        one_entry = json_object['review_id'] + "," +\
                    json_object['user_id'] + "," +\
                    json_object['business_id'] + "," +\
                    str(json_object['stars']) + "," +\
                    '\"'+ json_object['text'].replace("\"", " ") + '\"' +"," +\
                    json_object['date'] + "\n"
        csv_file.write(one_entry)
#         print("review_id: ", json_object["review_id"], " user_id: ", json_object["user_id"], " business_id: ", json_object["business_id"], " stars: ", json_object["stars"], " date: ", json_object["date"]);
        line = review_json.readline()
csv_file.close()

In [2]:
df = pd.read_csv('review_extracted.csv')
df.head()

,review_id,user_id,business_id,stars,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


## Dataset Sampling

In [3]:
print(len(df))
df_sampled = df.sample(frac=0.01,random_state=1)
print(len(df_sampled))

6685900
66859


# Remove Stopwords

In [4]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 'use']) # you can add more words into it to filter out other words
df_text_list = df_sampled['text'].tolist()
print(len(df_text_list))

66859


## Remove keywords from the list

In [5]:
# Remove new line characters
data = [re.sub('\s+', ' ', str(sent)) for sent in df_text_list]
data = [re.sub("\'", "", sent) for sent in data]

<>:2: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<ipython-input-5-85a3d6b662ad>:2: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', str(sent)) for sent in df_text_list]


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc

In [7]:
# Tokenize words and clean up text
data_words = list(sent_to_words(data))

In [8]:
print (data_words[0])
print (df_text_list[0])

['love', 'this', 'location', 'and', 'have', 'been', 'customer', 'for', 'well', 'over', 'year', 'have', 'always', 'had', 'pleasant', 'experience', 'and', 'the', 'staff', 'are', 'always', 'very', 'friendly']
I love this location and have been a customer for well over a year!  I have always had a pleasant experience and the staff are always very friendly.


## 9 Creating Bigram and Trigram Models

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/home/ubuntu/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['love', 'this', 'location', 'and', 'have', 'been', 'customer', 'for', 'well', 'over', 'year', 'have', 'always', 'had', 'pleasant', 'experience', 'and', 'the', 'staff', 'are', 'always', 'very', 'friendly']


In [10]:
bigram_mod[data_words[0]]
data_words[0]

['love',
 'this',
 'location',
 'and',
 'have',
 'been',
 'customer',
 'for',
 'well',
 'over',
 'year',
 'have',
 'always',
 'had',
 'pleasant',
 'experience',
 'and',
 'the',
 'staff',
 'are',
 'always',
 'very',
 'friendly']

## 10 Remove Stopwords, Make Bigrams and Lemmatize

In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

/home/ubuntu/anaconda3/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


## 11 Create the Dictionary and Corpus needed Topic Modeling

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [29]:
type(id2word)

gensim.corpora.dictionary.Dictionary

In [16]:
print(id2word.doc2bow(["food", "food", "food"]))
print(texts[1])

[(20, 3)]
['think', 'food', 'authentic', 'korean', 'food', 'everything', 'taste', 'fast', 'food', 'version', 'korean', 'food', 'food', 'complain', 'service', 'horrible', 'staff', 'member', 'seem', 'obedient', 'first', 'order', 'food', 'come', 'super', 'slowly', 'staff', 'member', 'seem', 'forget', 'ask', 'think', 'bad', 'korean_bbq', 'restaurant', 'ever', 'because', 'service', 'horrid', 'quality', 'food']


### save id2_word and data_lemmatized

In [35]:
# Save id2word
from gensim.test.utils import datapath
temp_file = datapath("id2words")
id2word.save(temp_file)

# Save texts
import pickle
pickle_out = open("parsed_texts.pickle","wb")
pickle.dump(texts, pickle_out)
pickle_out.close()


### load the saved data

In [45]:
id2word_loaded = gensim.corpora.dictionary.Dictionary.load(temp_file)

pickle_in = open("parsed_texts.pickle","rb")
texts_loaded = pickle.load(pickle_in)

i=0
print(texts_loaded[i])
print(texts[i])

['love', 'location', 'customer', 'well', 'year', 'always', 'pleasant', 'experience', 'staff', 'always', 'friendly']
['love', 'location', 'customer', 'well', 'year', 'always', 'pleasant', 'experience', 'staff', 'always', 'friendly']


## Load the existing Model

In [18]:
temp_file = '/home/ubuntu/anaconda3/lib/python3.7/site-packages/gensim/test/test_data/lda_1_percent'
lda_model = gensim.models.ldamodel.LdaModel.load(temp_file)

## 12. Building the Topic Model

In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## Save the model

In [87]:
from gensim.test.utils import datapath
temp_file = datapath("lda_1_percent")
lda_model.save(temp_file)

In [88]:
temp_file

'/home/ubuntu/anaconda3/lib/python3.7/site-packages/gensim/test/test_data/lda_1_percent'

## 13. View the topics in LDA model

In [19]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.119*"clean" + 0.022*"continue" + 0.022*"class" + 0.016*"incredibly" + '
  '0.014*"hop" + 0.014*"boba" + 0.013*"pour" + 0.012*"credit" + 0.012*"spa" + '
  '0.012*"tax"'),
 (1,
  '0.075*"room" + 0.035*"hotel" + 0.029*"kid" + 0.026*"parking" + 0.020*"area" '
  '+ 0.016*"pool" + 0.014*"plan" + 0.014*"brunch" + 0.013*"park" + '
  '0.013*"bathroom"'),
 (2,
  '0.174*"good" + 0.126*"food" + 0.043*"restaurant" + 0.043*"eat" + '
  '0.040*"price" + 0.030*"pretty" + 0.022*"burger" + 0.022*"fresh" + '
  '0.020*"everything" + 0.018*"menu"'),
 (3,
  '0.093*"tell" + 0.082*"call" + 0.036*"pay" + 0.030*"charge" + 0.030*"money" '
  '+ 0.025*"later" + 0.023*"receive" + 0.021*"issue" + 0.019*"rude" + '
  '0.018*"fix"'),
 (4,
  '0.088*"car" + 0.072*"favorite" + 0.047*"taco" + 0.031*"pork" + '
  '0.025*"mexican" + 0.021*"burrito" + 0.019*"authentic" + 0.018*"unique" + '
  '0.018*"salsa" + 0.018*"bean"'),
 (5,
  '0.100*"s" + 0.052*"beer" + 0.043*"selection" + 0.032*"cool" + 0.031*"that" '
  '+ 0.030

In [20]:
doc_lda[1][2]

[(7, [(14, 2.0)]),
 (10, [(7, 1.0)]),
 (11, [(4, 0.99999994)]),
 (12, [(7, 1.0)]),
 (13, [(15, 1.0)]),
 (14, [(7, 0.9851072)]),
 (15, [(15, 1.0)]),
 (16, [(2, 0.34551647), (7, 0.6544835)]),
 (17, [(2, 0.95163476), (12, 0.048365228)]),
 (18, [(2, 0.86983544), (14, 0.13016452)]),
 (19, [(7, 1.0)]),
 (20, [(2, 7.0)]),
 (21, [(11, 1.0)]),
 (22, [(3, 0.99999994)]),
 (23, [(15, 1.0)]),
 (24, [(15, 2.0)]),
 (25, [(15, 0.99999994)]),
 (26, [(15, 2.0)]),
 (27, [(15, 0.9755508)]),
 (28, [(13, 0.9999999)]),
 (29, [(11, 1.0)]),
 (30, [(2, 1.0)]),
 (31, [(7, 2.0000002)]),
 (32, [(14, 1.99427)]),
 (33, [(15, 1.0)]),
 (34, [(2, 1.0)]),
 (35, [(2, 0.87831205), (6, 0.031960305), (16, 0.08972765)]),
 (36, [(7, 2.0)]),
 (37, [(15, 1.0)])]

In [21]:
lda_model

## Testing on the unseen data

In [99]:
df_sampled2 = df.sample(frac=0.01,random_state=2)
df_sampled2.head(3)

,review_id,user_id,business_id,stars,text,date
3857412,7oeO1IBLpqxmd9aNcBNMTA,m-LlVyTuMIWmUUIQxHItfg,4EaqFG4v0A8nPEkbQwi6Dw,2.0,This is the 2nd time I have gone to Meineke fo...,2016-04-07 01:18:20
6442705,3ECqq0ZjU37K5zq75e78kg,3L5VEHgrphZkz1APeVdVDg,fL-b760btOaGa85OJ9ut3w,5.0,I haven't had BBQ this good in years. We did ...,2015-03-29 16:46:29
962501,dAC0P9RZkdD6EcoiCT2GmQ,l-B4axUNxQ9FVVxeQm8t2A,RZWx7pIsINH6nVqW7ys9cg,4.0,Very good down-home cooking. I had the fried t...,2014-06-03 22:32:46


In [98]:
df_sampled.head(3)

,review_id,user_id,business_id,stars,text,date
4634629,xw37uw9b2wFwVRLXSCtgNg,tIau2r-ox1LhC38wXX6oew,IUoj0ORP1VAV9xx4dmHP9g,5.0,I love this location and have been a customer ...,2018-09-07 20:41:56
4349124,jKxK47dz6D-ZpKqmd-oTeA,q0lV9MoRYDhyoLAQ9X91iA,hihud--QRriCYZw1zZvW4g,1.0,I think that the food is not at all authentic ...,2018-01-03 03:57:16
6056428,boaG5qtsdwR3KBwQbvnfAg,kUE8PMxx2_HTT32T16Z8Aw,zT3IxSfolhu4qU0tV0pTjg,2.0,If you schedule your appointment late in the a...,2018-06-27 01:40:52


In [103]:
def getReultsOnUnseenData(unseen_docs, lda_model):
    corpus = [id2word.doc2bow(text) for text in texts]
    lda_model.update(corpus)
    return lda_model[corpus]

In [101]:
def textPreprocessing(texts):
    data = [re.sub('\s+', ' ', str(sent)) for sent in texts]
    data = [re.sub("\'", "", sent) for sent in data]
    data_words = list(sent_to_words(data))
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    return data_lemmatized

In [102]:
unseen_docs = textPreprocessing(df_sampled2['text'].tolist())

/home/ubuntu/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [106]:
unseen_docs_res_distri = getReultsOnUnseenData(unseen_docs=unseen_docs, lda_model=lda_model)

unseen_docs_res_distri[0][0]

## Evaluation on Clustering

In [112]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.730780622207981

Coherence Score:  0.3878753024460627


## Visualization Part

In [22]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.290561  0.183307       1        1  26.560078
12    -0.159807 -0.003809       2        1  10.229616
14    -0.186220  0.103855       3        1   9.034567
13    -0.179876  0.021847       4        1   7.851896
2     -0.160941 -0.234728       5        1   7.444825
16    -0.076445 -0.293944       6        1   4.451869
17     0.006039  0.019147       7        1   4.176171
11     0.039123 -0.035903       8        1   3.666987
19     0.075875  0.023760       9        1   3.089801
3      0.053935  0.076002      10        1   2.886741
1      0.004507  0.128930      11        1   2.638813
6      0.048108 -0.161066      12        1   2.609333
5      0.088925  0.014953      13        1   2.380888
10     0.099892  0.013629      14        1   2.123738
18     0.103192  0.016066      15        1   2.107202
9      0.100164  0.042226      16        1   2.037342
15     0.111830  0.035533      17        1   1.803260
8      0.100354  0.022593      18        1   1.787620
4      0.098385 -0.000028      19        1   1.726321
0      0.123519  0.027629      20        1   1.392923, topic_info=     Category          Freq         Term         Total  loglift  logprob
79    Default  46206.000000         good  46206.000000  30.0000  30.0000
224   Default  36597.000000        place  36597.000000  29.0000  29.0000
20    Default  33550.000000         food  33550.000000  28.0000  28.0000
211   Default  52923.000000          not  52923.000000  27.0000  27.0000
127   Default  27013.000000        great  27013.000000  26.0000  26.0000
32    Default  25584.000000      service  25584.000000  25.0000  25.0000
28    Default  22589.000000        order  22589.000000  24.0000  24.0000
49    Default  40855.000000          get  40855.000000  23.0000  23.0000
78    Default  38729.000000           go  38729.000000  22.0000  22.0000
168   Default  31298.000000           do  31298.000000  21.0000  21.0000
63    Default  27884.000000         time  27884.000000  20.0000  20.0000
269   Default  27682.000000        would  27682.000000  19.0000  19.0000
41    Default  27647.000000           be  27647.000000  18.0000  18.0000
5     Default  14427.000000         love  14427.000000  17.0000  17.0000
745   Default   9555.000000         tell   9555.000000  16.0000  16.0000
262   Default  16733.000000          try  16733.000000  15.0000  15.0000
361   Default   8563.000000         feel   8563.000000  14.0000  14.0000
107   Default   8501.000000            s   8501.000000  13.0000  13.0000
379   Default  15875.000000       really  15875.000000  12.0000  12.0000
14    Default  26187.000000         come  26187.000000  11.0000  11.0000
781   Default   8438.000000         call   8438.000000  10.0000  10.0000
689   Default   9367.000000        drink   9367.000000   9.0000   9.0000
66    Default  14881.000000         also  14881.000000   8.0000   8.0000
30    Default  11333.000000   restaurant  11333.000000   7.0000   7.0000
73    Default  11378.000000          eat  11378.000000   6.0000   6.0000
81    Default  15899.000000         have  15899.000000   5.0000   5.0000
210   Default  11765.000000         nice  11765.000000   4.0000   4.0000
334   Default  10529.000000        price  10529.000000   3.0000   3.0000
479   Default   7120.000000         room   7120.000000   2.0000   2.0000
149   Default  19196.000000         back  19196.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
1175  Topic20   5919.633301        clean   5920.570312   4.2736  -2.1296
1755  Topic20   1103.044556     continue   1103.981445   4.2729  -3.8098
2630  Topic20   1075.394897        class   1076.331787   4.2729  -3.8351
1541  Topic20    806.327942   incredibly    807.264832   4.2726  -4.1231
1870  Topic20    702.240723          hop    703.177612   4.2724  -4.2613
4953  Topic20    677.578186         boba    678.515076   4.2724  -4.